# Test ADJ Connection to ESPORTS JSON Repository

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cx_Oracle
import os
import pandas as pd

In [2]:
!pip install dataprep
!pip install dask
!pip install pandas_profiling
!pip install -q scikit-learn
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: setuptools in /home/opc/.local/lib/python3.7/site-packages (59.4.0)
Requirement already up-to-date: wheel in /home/opc/.local/lib/python3.7/site-packages (0.37.0)
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: mxnet<2.0.0 in /home/opc/.local/lib/python3.7/site-packages (1.8.0.post0)
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object



def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [5]:
print(os.environ['TNS_ADMIN'])

/home/opc/adj_esportsdb


In [6]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

/home/opc/adj_esportsdb
Connection successful.
['1v1_model', 'match', 'match_detail', 'matchups', 'predictor', 'summoner']


In [7]:
data = db.open_collection('predictor')
all_data = list()
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)

print('Data length: {}'.format(len(all_data)))

Data length: 611650


In [8]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)


,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,1,0,500,32,0,0,0,0,0,0,...,0,100,0,0,570,31,0,0,1,0
1,1,0,500,32,0,0,0,0,0,0,...,0,100,0,0,490,30,0,0,1,0
2,1,0,500,30,0,0,0,0,0,0,...,0,100,0,0,600,28,0,0,1,0
3,1,0,500,32,0,0,0,0,0,0,...,0,100,0,0,576,39,0,0,1,0
4,0,0,500,28,0,0,0,0,0,0,...,0,100,0,0,520,36,0,0,1,0


In [9]:
df.describe()

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
count,611650.0,611650.000000,611650.000000,611650.000000,6.116500e+05,611650.000000,611650.000000,611650.000000,6.116500e+05,611650.0,...,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.000000,611650.0
mean,0.5,421.478162,5342.874812,45.010455,1.036672e+05,6.395738,16.402603,0.054366,8.677962e+05,0.0,...,28625.628209,141.173133,2874.655939,2662.621507,1375.802626,69.177343,5897.540322,48969.268879,8.386198,0.0
std,0.5,743.480280,3918.915224,20.955460,1.410567e+05,12.325412,34.858512,0.995448,5.614745e+05,0.0,...,40392.234214,44.568597,3455.771227,4747.649685,621.051844,38.468999,4623.942694,52861.411627,4.417011,0.0
min,0.0,0.000000,500.000000,-10.000000,0.000000e+00,-30.000000,0.000000,0.000000,0.000000e+00,0.0,...,0.000000,51.000000,0.000000,0.000000,340.000000,-14.000000,0.000000,0.000000,1.000000,0.0
25%,0.0,0.000000,2053.000000,33.000000,1.133000e+04,0.000000,0.000000,0.000000,4.200990e+05,0.0,...,3003.000000,112.000000,389.000000,80.000000,890.000000,45.000000,2020.000000,8612.000000,5.000000,0.0
50%,0.5,150.000000,4675.000000,38.000000,5.554100e+04,0.000000,0.000000,0.000000,8.402300e+05,0.0,...,10107.000000,125.000000,1659.000000,791.000000,1291.000000,59.000000,5151.000000,29283.000000,9.000000,0.0
75%,1.0,512.000000,7862.000000,47.000000,1.396210e+05,7.000000,10.000000,0.000000,1.260449e+06,0.0,...,38999.750000,155.000000,4169.000000,3015.000000,1724.000000,81.000000,8957.000000,75776.750000,12.000000,0.0
max,1.0,18181.000000,28341.000000,519.000000,2.498251e+06,82.000000,278.000000,60.000000,3.147362e+06,0.0,...,430621.000000,699.000000,44368.000000,82476.000000,7534.000000,921.000000,33822.000000,541897.000000,18.000000,0.0


In [10]:
from pandas_profiling import ProfileReport

In [ ]:
report = ProfileReport(df)
# report #uncomment to display all.

In [12]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [13]:
df = TabularDataset(df)

# drop columns we don't want
df = df.drop(columns=['identifier'])

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head(5)

,winner,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
556492,1,165,932,30,0,0,0,0,240067,0,...,563,100,54,156,593,30,620,929,2,0
452181,0,152,6212,33,0,0,4,0,1140326,0,...,56337,138,1653,1339,1361,57,6555,64037,10,0
36066,0,0,500,26,0,0,0,0,0,0,...,0,100,0,0,530,17,0,0,1,0
500051,0,1030,11252,50,352683,0,10,0,1860656,0,...,19840,139,9029,16582,2225,96,15496,149976,16,0
158171,1,120,3468,42,65081,0,56,0,480120,0,...,2647,108,1420,1894,893,33,3154,35342,6,0


In [14]:
label = 'winner'

In [15]:
save_path = './autogluon_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./autogluon_trained_models/"
AutoGluon Version:  0.3.1
Train Data Rows:    489320
Train Data Columns: 43
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    19340.55 MB
	Train Data (Original)  Memory Usage: 168.33 MB (0.9% of available memory)
	

In [16]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head(5)

,trueDamageTaken,totalGold,magicResist,timeEnemySpentControlled,ccReduction,goldPerSecond,spellVamp,timestamp,bonusArmorPenPercent,magicPenPercent,...,physicalDamageDone,attackSpeed,magicDamageTaken,magicDamageDoneToChampions,healthMax,armor,xp,totalDamageDone,level,armorPen
0,0,500,32,0,0,0,0,0,0,0,...,0,100,0,0,570,31,0,0,1,0
2,0,500,30,0,0,0,0,0,0,0,...,0,100,0,0,600,28,0,0,1,0
8,0,500,30,0,0,0,0,0,0,0,...,0,100,0,0,536,22,0,0,1,0
9,0,500,30,0,0,0,0,0,0,0,...,0,100,0,0,535,32,0,0,1,0
13,158,2320,44,0,10,0,0,540203,0,0,...,3350,132,1179,895,973,41,2796,13841,6,0


In [17]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Predictions:  
 0         0
2         0
8         1
9         1
13        0
         ..
611625    0
611641    0
611645    1
611648    0
611649    1
Name: winner, Length: 122330, dtype: int64


Evaluation: accuracy on test data: 0.8277609744134717
Evaluations on test data:
{
    "accuracy": 0.8277609744134717,
    "balanced_accuracy": 0.8277726847714724,
    "mcc": 0.6555552097984404,
    "f1": 0.8274760906589808,
    "precision": 0.8312604876122791,
    "recall": 0.823725995239803
}


In [18]:
predictor.leaderboard(test, silent=False)

                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.827761   0.837556       27.665979       1.128662  2609.017195                 0.174247                0.011962           2.785339            2       True         14
1        NeuralNetMXNet    0.814616   0.822231        3.415714       0.331388  2068.578531                 3.415714                0.331388        2068.578531            1       True         12
2      RandomForestGini    0.766296   0.767675        5.404150       0.217736   176.791164                 5.404150                0.217736         176.791164            1       True          5
3      RandomForestEntr    0.765773   0.768901       11.371829       0.216782   282.099415                11.371829                0.216782         282.099415            1       True          6
4        ExtraTreesEntr    0.7

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.827761,0.837556,27.665979,1.128662,2609.017195,0.174247,0.011962,2.785339,2,True,14
1,NeuralNetMXNet,0.814616,0.822231,3.415714,0.331388,2068.578531,3.415714,0.331388,2068.578531,1,True,12
2,RandomForestGini,0.766296,0.767675,5.404150,0.217736,176.791164,5.404150,0.217736,176.791164,1,True,5
3,RandomForestEntr,0.765773,0.768901,11.371829,0.216782,282.099415,11.371829,0.216782,282.099415,1,True,6
4,ExtraTreesEntr,0.765029,0.770535,3.162656,0.115001,38.284721,3.162656,0.115001,38.284721,1,True,9
5,ExtraTreesGini,0.764972,0.770127,4.050691,0.217266,36.846248,4.050691,0.217266,36.846248,1,True,8
6,NeuralNetFastAI,0.662454,0.662852,2.379005,0.103901,604.734415,2.379005,0.103901,604.734415,1,True,10
7,XGBoost,0.636696,0.634450,0.343345,0.032097,6.617191,0.343345,0.032097,6.617191,1,True,11
8,LightGBM,0.635649,0.632203,0.504502,0.040395,33.061020,0.504502,0.040395,33.061020,1,True,4
9,LightGBMXT,0.624393,0.626073,0.484653,0.059846,35.053180,0.484653,0.059846,35.053180,1,True,3


In [19]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 43 features using 1000 rows with 3 shuffle sets...
	4449.2s	= Expected runtime (1483.07s per shuffle set)
	144.23s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
totalGold,0.224333,0.014844,0.000728,3,0.309389,0.139278
timestamp,0.216333,0.014189,0.000715,3,0.297639,0.135028
xp,0.151000,0.012000,0.001049,3,0.219761,0.082239
totalDamageTaken,0.113667,0.023245,0.006828,3,0.246864,-0.019530
magicDamageTaken,0.109667,0.023671,0.007589,3,0.245306,-0.025973
minionsKilled,0.109333,0.012503,0.002166,3,0.180979,0.037688
physicalDamageTaken,0.109333,0.009452,0.001241,3,0.163492,0.055174
totalDamageDoneToChampions,0.095000,0.012767,0.002983,3,0.168157,0.021843
participantId,0.092333,0.010599,0.002182,3,0.153065,0.031601
trueDamageDoneToChampions,0.091667,0.001155,0.000026,3,0.098283,0.085050
